In [137]:
import numpy as np
from PIL import Image
import cupy as cp
import time
import pandas as pd
from scipy.signal import medfilt2d

In [138]:
img = Image.open('Noise_salt_and_pepper.bmp')
arr = cp.array(img)
print(arr.shape)

(428, 320)


In [139]:
class CudaSaltAndPepper:
    def __init__(self, matrix_size: tuple, matrix: cp.ndarray, pars: dict):
        self.add_kernel = cp.RawKernel(r'''
                extern "C" 
                __global__ void saltandpepper(unsigned char* input, unsigned char* output, int width, int height) {
                    int x = blockIdx.x * blockDim.x + threadIdx.x;
                    int y = blockIdx.y * blockDim.y + threadIdx.y;

                    if (x < width && y < height) {
                        unsigned char window[9];
                        int index = 0;

                        for (int i = -1; i <= 1; i++) {
                            for (int j = -1; j <= 1; j++) {
                                int nx = x + i;
                                int ny = y + j;

                                if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                                    //printf("%u", input[ny * width + nx]);
                                    window[index] = input[ny * width + nx];
                                } else {
                                    window[index] = 0;
                                }

                                index++;
                            }
                        }

                        for (int i = 0; i < 9; i++) {
                            for (int j = i + 1; j < 9; j++) {
                                if (window[i] > window[j]) {
                                    unsigned char temp = window[i];
                                    window[i] = window[j];
                                    window[j] = temp;
                                }
                            }
                        }

                        output[y * width + x] = window[4];
                    }
                }
            ''',
            "saltandpepper")
        self.shape = matrix_size
        self.matrix = cp.asarray(matrix.flatten())
        self.matrix_scipy = cp.asarray(matrix)
        self.matrix_func = cp.asarray(matrix)
        self.params = pars
        self.result = cp.zeros((self.shape[0] * self.shape[1]), dtype=cp.uint8)

    def salt_and_pepper(self):
        output = np.zeros(self.shape, dtype=np.uint8)
        window = []
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                if (i <= 0 or j <= 0 or i >= self.shape[0] - 1 or j >= self.shape[1] - 1):
                    continue
                window = [self.matrix_func[i+x, j+y] for x in range(-1, 2) for y in range(-1, 2)]
                '''window = [
                    self.matrix_func[i-1, j-1],
                    self.matrix_func[i, j-1],
                    self.matrix_func[i+1, j-1],
                    self.matrix_func[i-1, j],
                    self.matrix_func[i, j],
                    self.matrix_func[i+1, j],
                    self.matrix_func[i-1, j+1],
                    self.matrix_func[i, j+1],
                    self.matrix_func[i+1, j+1],
                ]'''
                window.sort()
                output[i][j] = window[4]
                window = []
        return output

    def get_result(self):
        gstart = time.perf_counter()
        result_gpu = self.add_kernel((self.params["blockX"], self.params["blockY"]),
                                    (self.params["gridX"], self.params["gridY"]),
                                    (self.matrix, self.result,
                                    self.shape[1], self.shape[0]))
        gend = time.perf_counter()

        scipystart = time.perf_counter()
        result_scipy = medfilt2d(self.matrix_scipy.get(), kernel_size=3)
        scipyend = time.perf_counter()

        cfuncstart = time.perf_counter()
        result_cpu_func = self.salt_and_pepper()
        cfuncend = time.perf_counter()

        img_gpu = Image.fromarray(self.result.get().reshape(self.shape))
        img_scipy = Image.fromarray(result_scipy)
        img_func = Image.fromarray(result_cpu_func)

        img_gpu.save('out_gpu.bmp')
        img_scipy.save('out_scipy.bmp')
        img_func.save('out_func.bmp')

        return {
            "matrix size": str(self.shape),
            "parameters": str(self.params),
            "gpu time": (gend - gstart),
            "scipy time": (scipyend - scipystart),
            "func time": (cfuncend - cfuncstart),
            "gpu result": "![Alt Text](out_gpu.bmp)",
            "scipy result": "![Alt Text](out_scipy.bmp)",
            "func result": "![Alt Text](out_func.bmp)",
        }

In [140]:
size = arr.shape
block = (32, 32)
grid = (round((size[1] + block[0] - 1) / block[0]), round((size[0] + block[1] - 1) / block[1]))

obj = CudaSaltAndPepper(size,
                        arr,
                        {
                            "blockX": block[0],
                            "blockY": block[1],
                            "gridX": grid[0],
                            "gridY": grid[1],
                        })

result = obj.get_result()

df1 = pd.DataFrame(result, index=[0])
print(df1.to_markdown(index=False))

| matrix size   | parameters                                             |    gpu time |   scipy time |   func time | gpu result               | scipy result               | func result               |
|:--------------|:-------------------------------------------------------|------------:|-------------:|------------:|:-------------------------|:---------------------------|:--------------------------|
| (428, 320)    | {'blockX': 32, 'blockY': 32, 'gridX': 11, 'gridY': 14} | 5.67001e-05 |    0.0090728 |      195.43 | ![Alt Text](out_gpu.bmp) | ![Alt Text](out_scipy.bmp) | ![Alt Text](out_func.bmp) |


In [142]:
res = pd.DataFrame({
    "gpu vs func":  (df1["func time"] / df1["gpu time"]).apply('{:.2f}'.format),
    "gpu vs scipy": (df1["scipy time"] / df1["gpu time"]).apply('{:.2f}'.format),
})
print(res.to_markdown(index=False))

|   gpu vs func |   gpu vs scipy |
|--------------:|---------------:|
|   3.44673e+06 |         160.01 |
